# SNEWS Firedrill Test May 25th


## Installing Stuff

- [**SNEWS_PT Repo**](https://github.com/SNEWS2/SNEWS_Publishing_Tools) <br>
- [**hop repo**](https://github.com/scimma/hop-client) <br>
- [**hop auth**](https://my.hop.scimma.org/hopauth/)


Start by making sure your hop credentials are updated for firedrills:

snews.alert-firedrill
snews.experiments-firedrill
snews.mma-firedrill

- Make sure you are using hop 0.5.0

```pip install -U hop-client``` <br>
You can check your version with `hop --version`

Install SNEWS_PT

```pip install snews-pt```





# Detector Names (PT Format)

<font color='red'>
"Baksan" <br>
"Borexino"<br>
"DS-20K"<br>
"DUNE"<br>
"HALO"<br>
"HALO-1kT"<br>
"Hyper-K"<br>
"ICE"<br>
"JUNO"<br>
"KM3NeT"<br>
"KamLAND"<br>
"LVD"<br>
"LZ"<br>
"MicroBooNe"<br>
"NOvA"<br>
"PandaX-4T"<br>
"SBND"<br>
"SNO+"<br>
"Super-K"<br>
"XENONnT"<br>
    </font> <br>
    and the default name <br>
<font color='blue'>"TEST"</font>

## Using the API

In [4]:
from snews_pt.snews_pub import SNEWSTiersPublisher
from datetime import  datetime
from time import sleep


def test_time():
    # return datetime.utcnow().strftime("%y/%m/%d %H:%M:%S:%f")
    return datetime.utcnow().isoformat()

In [10]:
message = SNEWSTiersPublisher(detector_name="KamLAND",
                              machine_time=test_time() ,
                              neutrino_time=test_time(),
                              p_val=0.08,
                              firedrill_mode=True)

message.send_to_snews()

# sleep(3)

# message2 = SNEWSTiersPublisher(detector_name="Borexino",
#                               machine_time=test_time() ,
#                               neutrino_time=test_time(),
#                               p_val=0.03,
#                               firedrill_mode=False)

# message2.send_to_snews()


----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-test
_id                 :4_CoincidenceTier_2022-09-28T14:28:52.193756
detector_name       :KamLAND
machine_time        :2022-09-28T14:28:52.193756
neutrino_time       :2022-09-28T14:28:52.193763
p_val               :0.08
meta                :{}
schema_version      :1.1.1
sent_time           :2022-09-28T14:28:52.193771
----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-test
_id                 :7_CoincidenceTier_2022-09-28T14:28:56.603930
detector_name       :Borexino
machine_time        :2022-09-28T14:28:56.603930
neutrino_time       :2022-09-28T14:28:56.603957
p_val               :0.03
meta                :{}
schema_version      :1.1.1
sent_time           :2022-09-28T14:28:56.603975


### Submit from JSON

Let's submit an observation message from a JSON file. For this we use the `"coincidence_example.json"` which contains
```JSON
{
    "machine_time": 
        "22/06/14 14:57:238275",
    "neutrino_time": 
        "22/06/14 14:50:345275",
    "p_val":
        0.07,
    "something extra":
        "Our neutrino time is defined as the first event above bg"
}
```

In [ ]:
json_message = SNEWSTiersPublisher.from_json("coincidence_example.json")
json_message.send_to_snews()

It is also possible to modify the content of the json file.

In [ ]:
json_message = SNEWSTiersPublisher.from_json("coincidence_example.json", detector_name="ICE")
json_message.send_to_snews()

## Using the CLI






We also provide Command Line Interface. CLI allows subscribing and publishing messages using the terminal. <br>
Display the complete help with
```bash
snews_pt --help
```

Subscribe to alerts
```bash
snews_pt subscribe
```

Publish JSON files
```bash
snews_pt publish coincidence_example.json
```

**Bonus**:<br> If you have a follow-up script which uses the alert content and does more calculations, you can plug this script to subscribe function
```bash
snews_pt subscribe -p mycustomscript.py
```

> The `mycustomscript.py` should contain 
> ```python
> import sys, json
> data = json.load(open(sys.argv[1]))
> ``` 

> then it the `data` contains the alert content as a dictionary object.